# XGBoost — Optimización avanzada y balanceo

Este notebook:

- Buscar hiperparámetros con `GridSearchCV` y `RandomizedSearchCV` para `XGBClassifier`.
- Aplicar técnicas de balanceo (`SMOTE`, `ADASYN`, y `scale_pos_weight`).
- Implementar **focal loss** como función objetivo personalizada para XGBoost.
- Documentar mejoras en **PR-AUC** y **F1-Score**.
- Guardar el mejor modelo entrenado.

> Reemplaza la carga de datos por tu propio `data.csv` o usa el dataset de ejemplo incluido en la celda de `Carga de datos`.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, auc, f1_score, average_precision_score, classification_report
from sklearn.utils import compute_class_weight

from xgboost import XGBClassifier, DMatrix, train as xgb_train
import xgboost as xgb

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline as ImbPipeline

import joblib


In [2]:

# --- CARGA DE DATOS ---
# Opción A: cargar tu CSV (reemplaza 'data.csv' por tu ruta)
csv_path = 'data.csv'  # <-- cambia si quieres
try:
    df = pd.read_csv(csv_path)
    print(f'Datos cargados desde {csv_path} — shape:', df.shape)
except Exception as e:
    print('No se encontró data.csv — se cargará un dataset de ejemplo (make_classification).\n', e)
    from sklearn.datasets import make_classification
    X_s, y_s = make_classification(n_samples=10000, n_features=20, n_informative=6, n_redundant=2, 
                                   n_clusters_per_class=2, weights=[0.95], flip_y=0.01, random_state=42)
    df = pd.DataFrame(X_s, columns=[f'feat_{i}' for i in range(X_s.shape[1])])
    df['target'] = y_s
    print('Dataset de ejemplo creado — shape:', df.shape)

# Asume que la columna objetivo se llama 'target'. Si tiene otro nombre, cámbialo aquí:
target_col = 'target'
if target_col not in df.columns:
    raise ValueError(f'No se encontró la columna objetivo "{target_col}" en el dataframe.')

# Separar X, y
X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

print('Distribución de clases:\n', y.value_counts())


No se encontró data.csv — se cargará un dataset de ejemplo (make_classification).
 [Errno 2] No such file or directory: 'data.csv'
Dataset de ejemplo creado — shape: (10000, 21)
Distribución de clases:
 target
0    9454
1     546
Name: count, dtype: int64


In [3]:

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Scaling (se puede integrar en el pipeline)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Shapes:', X_train.shape, X_test.shape)


Shapes: (8000, 20) (2000, 20)


In [4]:

def evaluate_model(clf, X_test, y_test, verbose=True):
    y_proba = clf.predict_proba(X_test)[:,1]
    y_pred = clf.predict(X_test)
    pr_auc = average_precision_score(y_test, y_proba)
    f1 = f1_score(y_test, y_pred)
    if verbose:
        print('PR-AUC (Average Precision):', round(pr_auc,4))
        print('F1-score:', round(f1,4))
        print('\nClassification report:\n', classification_report(y_test, y_pred))
    return {'pr_auc': pr_auc, 'f1': f1, 'y_proba': y_proba, 'y_pred': y_pred}


In [5]:

# Baseline XGBoost (sin balanceo ni búsqueda de hiperparámetros)
baseline = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=4)
baseline_pipeline = Pipeline([('scaler', StandardScaler()), ('xgb', baseline)])
baseline_pipeline.fit(X_train, y_train)
print('Baseline evaluacion:')
baseline_results = evaluate_model(baseline_pipeline, X_test, y_test)


Baseline evaluacion:
PR-AUC (Average Precision): 0.8014
F1-score: 0.7303

Classification report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1891
           1       0.94      0.60      0.73       109

    accuracy                           0.98      2000
   macro avg       0.96      0.80      0.86      2000
weighted avg       0.98      0.98      0.97      2000



In [6]:

# --- GRID SEARCH (ejemplo acotado) ---
param_grid = {
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__subsample': [0.6, 0.8, 1.0],
    'xgb__colsample_bytree': [0.6, 0.8, 1.0],
    'xgb__n_estimators': [100, 300]
}

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=4)

grid_pipeline = Pipeline([('scaler', StandardScaler()), ('xgb', xgb_clf)])

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

grid_search = GridSearchCV(grid_pipeline, param_grid, scoring='average_precision', cv=cv, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print('Mejor combo GridSearchCV:', grid_search.best_params_)
best_grid = grid_search.best_estimator_
grid_results = evaluate_model(best_grid, X_test, y_test)


Fitting 3 folds for each of 108 candidates, totalling 324 fits


/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:56:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:56:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:56:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:56:52] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-pack

Mejor combo GridSearchCV: {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 7, 'xgb__n_estimators': 300, 'xgb__subsample': 0.8}
PR-AUC (Average Precision): 0.8191
F1-score: 0.7126

Classification report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1891
           1       0.95      0.57      0.71       109

    accuracy                           0.97      2000
   macro avg       0.96      0.78      0.85      2000
weighted avg       0.97      0.97      0.97      2000



In [7]:

# --- RANDOMIZED SEARCH (más amplio) ---
from scipy.stats import randint, uniform

param_dist = {
    'xgb__max_depth': randint(3, 10),
    'xgb__learning_rate': uniform(0.01, 0.3),
    'xgb__subsample': uniform(0.5, 0.5),
    'xgb__colsample_bytree': uniform(0.5, 0.5),
    'xgb__n_estimators': randint(100, 500),
    'xgb__gamma': uniform(0, 5)
}

rand_search = RandomizedSearchCV(grid_pipeline, param_dist, n_iter=30, scoring='average_precision', cv=cv, n_jobs=-1, random_state=42, verbose=1)
rand_search.fit(X_train, y_train)

print('Mejor combo RandomizedSearchCV:', rand_search.best_params_)
best_rand = rand_search.best_estimator_
rand_results = evaluate_model(best_rand, X_test, y_test)


Fitting 3 folds for each of 30 candidates, totalling 90 fits


/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:57:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:57:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:57:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:57:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/bianca_cal/.local/lib/python3.10/site-pack

Mejor combo RandomizedSearchCV: {'xgb__colsample_bytree': 0.6467440873590191, 'xgb__gamma': 0.07039911357542228, 'xgb__learning_rate': 0.06965272122664154, 'xgb__max_depth': 5, 'xgb__n_estimators': 388, 'xgb__subsample': 0.8029799873905057}
PR-AUC (Average Precision): 0.8299
F1-score: 0.6982

Classification report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99      1891
           1       0.98      0.54      0.70       109

    accuracy                           0.97      2000
   macro avg       0.98      0.77      0.84      2000
weighted avg       0.97      0.97      0.97      2000



In [8]:

# --- BALANCEO: SMOTE y ADASYN ---
smote = SMOTE(random_state=42)
adasyn = ADASYN(random_state=42)

xgb_clf2 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=4, n_estimators=300)

# Pipeline con SMOTE
pipe_smote = ImbPipeline([('smote', smote), ('scaler', StandardScaler()), ('xgb', xgb_clf2)])
pipe_smote.fit(X_train, y_train)
print('Evaluación con SMOTE:')
smote_results = evaluate_model(pipe_smote, X_test, y_test)

# Pipeline con ADASYN
pipe_adasyn = ImbPipeline([('adasyn', adasyn), ('scaler', StandardScaler()), ('xgb', xgb_clf2)])
pipe_adasyn.fit(X_train, y_train)
print('Evaluación con ADASYN:')
adasyn_results = evaluate_model(pipe_adasyn, X_test, y_test)


Evaluación con SMOTE:
PR-AUC (Average Precision): 0.8265
F1-score: 0.794

Classification report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      1891
           1       0.88      0.72      0.79       109

    accuracy                           0.98      2000
   macro avg       0.93      0.86      0.89      2000
weighted avg       0.98      0.98      0.98      2000

Evaluación con ADASYN:
PR-AUC (Average Precision): 0.809
F1-score: 0.759

Classification report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      1891
           1       0.86      0.68      0.76       109

    accuracy                           0.98      2000
   macro avg       0.92      0.84      0.87      2000
weighted avg       0.98      0.98      0.98      2000



In [9]:

# --- Ajuste scale_pos_weight ---
n_pos = sum(y_train==1)
n_neg = sum(y_train==0)
scale = n_neg / (n_pos + 1e-6)
print('scale_pos_weight sugerido:', round(scale,3))

xgb_spw = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, scale_pos_weight=scale, n_estimators=300)
pipe_spw = Pipeline([('scaler', StandardScaler()), ('xgb', xgb_spw)])
pipe_spw.fit(X_train, y_train)
print('Evaluación con scale_pos_weight:')
spw_results = evaluate_model(pipe_spw, X_test, y_test)


scale_pos_weight sugerido: 17.307
Evaluación con scale_pos_weight:
PR-AUC (Average Precision): 0.8422
F1-score: 0.8105

Classification report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1891
           1       0.95      0.71      0.81       109

    accuracy                           0.98      2000
   macro avg       0.97      0.85      0.90      2000
weighted avg       0.98      0.98      0.98      2000



In [10]:

# --- FOCAL LOSS: objetivo personalizado para XGBoost (binario) ---
# Basado en la definición: FL(p_t) = - alpha * (1 - p_t)^gamma * log(p_t)
# Necesitamos gradiente y hessiano respecto al logit (raw pred).

import math

def focal_loss_obj(preds, dtrain, alpha=0.25, gamma=2.0):
    labels = dtrain.get_label()
    preds_sigmoid = 1.0 / (1.0 + np.exp(-preds))
    eps = 1e-9
    p = preds_sigmoid
    y = labels
    # p_t = p cuando y==1, else 1-p
    p_t = p * y + (1 - p) * (1 - y)
    alpha_factor = alpha * y + (1 - alpha) * (1 - y)
    # Gradient (aproximación utilizada en práctica)
    grad = alpha_factor * ( ( - (1 - p_t)**gamma ) * (y - p) ) + alpha_factor * ( - gamma * (1 - p_t)**(gamma-1) * ( - (y - p) ) * np.log(np.clip(p_t, eps, 1.0)) )
    # Hessian (aproximación para estabilidad)
    hess = np.clip(np.abs(grad) * 0.5 + 1e-6, 1e-12, 1e6)
    return grad, hess


In [11]:

# --- Entrenamiento con focal loss (API xgboost.train) ---
dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
dtest = xgb.DMatrix(X_test_scaled, label=y_test)

params = {
    'eta': 0.05,
    'max_depth': 5,
    'verbosity': 0,
    'objective': 'binary:logistic'
}

bst = xgb.train(params, dtrain, num_boost_round=200, obj=lambda preds, dtrain: focal_loss_obj(preds, dtrain, alpha=0.25, gamma=2.0))

y_prob_focal = bst.predict(dtest)
y_pred_focal = (y_prob_focal > 0.5).astype(int)

print('PR-AUC focal:', round(average_precision_score(y_test, y_prob_focal),4))
print('F1 focal:', round(f1_score(y_test, y_pred_focal),4))


PR-AUC focal: 0.8055
F1 focal: 0.6786


## Comparación de resultados

Registra los PR-AUC y F1-score obtenidos por cada aproximación (Baseline, GridSearch, RandomizedSearch, SMOTE, ADASYN, scale_pos_weight, Focal Loss) y guarda una tabla con los resultados para documentar las mejoras.

In [12]:

results = {
    'baseline': baseline_results,
    'grid_search': grid_results if 'grid_results' in globals() else None,
    'random_search': rand_results if 'rand_results' in globals() else None,
    'smote': smote_results if 'smote_results' in globals() else None,
    'adasyn': adasyn_results if 'adasyn_results' in globals() else None,
    'scale_pos_weight': spw_results if 'spw_results' in globals() else None,
    'focal': {'pr_auc': average_precision_score(y_test, y_prob_focal) if 'y_prob_focal' in globals() else None,
              'f1': f1_score(y_test, y_pred_focal) if 'y_prob_focal' in globals() else None}
}

rows = []
for k,v in results.items():
    if v is None:
        rows.append({'method': k, 'pr_auc': None, 'f1': None})
    else:
        rows.append({'method': k, 'pr_auc': v.get('pr_auc'), 'f1': v.get('f1')})
summary_df = pd.DataFrame(rows).sort_values('pr_auc', ascending=False)
summary_df


,method,pr_auc,f1
5,scale_pos_weight,0.842161,0.810526
2,random_search,0.829858,0.698225
3,smote,0.826461,0.793970
1,grid_search,0.819075,0.712644
4,adasyn,0.808983,0.758974
6,focal,0.805509,0.678571
0,baseline,0.801360,0.730337


In [13]:

# --- GUARDAR MEJOR MODELO ---
best_model = None
if 'best_rand' in globals():
    best_model = best_rand
elif 'best_grid' in globals():
    best_model = best_grid
elif 'pipe_smote' in globals():
    best_model = pipe_smote
else:
    best_model = baseline_pipeline

joblib.dump(best_model, 'best_xgb_model.joblib')
print('Mejor modelo guardado en: best_xgb_model.joblib')


Mejor modelo guardado en: best_xgb_model.joblib
